#Sentiment Classification using Keras and Word Embedding

### Uploading the data

In [9]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataset/data_capres.csv', encoding="ISO-8859-1")
df.head()

,ID,Tweet_ID,Isi_Tweet,Tanggal_Tweet,Kandidat,Aplikasi,Lokasi User,Sentimen
0,133343972,patih_patria,@FahrelStv Gak setuju Jokowi jadi Cawapres.. C...,8/1/2013 0:33,Jokowi,web,BLITAR - INDONESIA,1
1,921720487,RintisBautista,"capres jokowi,wacapres abraham samad. gubernur...",8/1/2013 4:11,Jokowi,"<a href=""http://janetter.net/"" rel=""nofollow"">...",Didalam hatinya recha :love,1
2,-2120434840,JokowiPres,"RT @RintisBautista: capres jokowi,wacapres abr...",8/1/2013 4:32,Jokowi,"<a href=""http://roundteam.co"" rel=""nofollow"">R...",NaN,1
3,-116006035,Franliiiii,capres ( Prabowo ) and cawapres ( jokowi ) and...,8/1/2013 7:33,Prabowo,web,Manado,1
4,-906300614,JokowiPres,RT @Franliiiii: capres ( Prabowo ) and cawapre...,8/1/2013 7:36,Prabowo,"<a href=""http://roundteam.co"" rel=""nofollow"">R...",NaN,1


In [ ]:
df = df[['Isi_Tweet', 'Sentimen']]
df.head()

,Isi_Tweet,Sentimen
0,@FahrelStv Gak setuju Jokowi jadi Cawapres.. C...,1
1,"capres jokowi,wacapres abraham samad. gubernur...",1
2,"RT @RintisBautista: capres jokowi,wacapres abr...",1
3,capres ( Prabowo ) and cawapres ( jokowi ) and...,1
4,RT @Franliiiii: capres ( Prabowo ) and cawapre...,1


In [ ]:
#load stopwords

stp = open('/content/drive/My Drive/Colab Notebooks/dataset/twitter_stp.dic')
stp_words = []
item = stp.readline()

while item != '':
  stp_words.append(item)
  item = stp.readline()

In [ ]:
import re
import string
from textblob import TextBlob

list_data = []

for i in df.Isi_Tweet:
  list_data.append(i)

list_data_bersih1 = []
list_data_bersih = []

import re
pat1 = r'@[A-Za-z0-9_]+' #deleting @mention
pat2 = r'https?://[A-Za-z0-9./]+' #delete url
pat3 = r'#[A-Za-z0-9]+' #delete hashtag
pat4 = r'\[A-Za-z0-9]+'#delete alphabeth+number
pat5 = r'\<[A-Za-z0-9+_>]+' #deleting error mp
pat6 = r'[0-9]+' #delete full of number with more than 1
pattern = re.compile('\W')  #deleting white space

combined_pat = r'|'.join((pat1, pat2, pat3,pat4, pat5, pat6)) #masking creating

for j in list_data:
  dataBersih1 = re.sub(combined_pat,'',str(j)) #filtering process
  list_data_bersih1.append(dataBersih1) #store cleaning data

for k in list_data_bersih1:   
  dataBersih2 = re.sub(pattern,' ',str(k)) #menghapus karakter non alphanumeric
  dataBersih3 = re.sub(r'\b\w{1,1}\b', '', dataBersih2) #mengapus karakter 1 digit
  list_data_bersih.append(dataBersih3) #menyimpan dalam list

In [ ]:
list_data_bersih

[' Gak setuju Jokowi jadi Cawapres   CAPRES JOKOWI harga mati   ',
 'capres jokowi wacapres abraham samad  gubernur ahok  gua yakin koruptor abissss  ',
 'RT   capres jokowi wacapres abraham samad  gubernur ahok  gua yakin koruptor abissss  ',
 'capres   Prabowo   and cawapres   jokowi   and gubDKI  Ahok  mantap lanjutkan     buat pak presiden  SBY  bubarkan saja FPI',
 'RT   capres   Prabowo   and cawapres   jokowi   and gubDKI  Ahok  mantap lanjutkan     buat pak presiden  SBY  bubarkan saja FPI',
 'jd skenarionya gini   biar Prabowo jd presiden  Jokowi tetepgubernur  kalo Jakarta berhasil gak usah nunggu  buat gantiin Prabowo',
 'SBY mantan TNI  dan Calon Presiden Prabowo subianto adalah Mantan KOPASSUS   Krn anggoto TNI disiplin   Maka dari itu  smw presiden TEGAS   ',
 ' mohon untuk pak Presiden Prabowo Subiyanto  tolong jangan jadikan kami  Perangkat Desa  sebagai tumbal kebijakan pemerintah ',
 'RT   SBY mantan TNI  dan Calon Presiden Prabowo subianto adalah Mantan KOPASSUS   Kr

In [10]:
list_sentimen =[]
for i in df.Sentimen:
  list_sentimen.append(i)

df_use = list(zip(list_data_bersih, list_sentimen))
df_use = pd.DataFrame(df_use, columns=['tweet', 'label'])
df_use.head()

,tweet,label
0,Gak setuju Jokowi jadi Cawapres CAPRES JOKO...,1
1,capres jokowi wacapres abraham samad gubernur...,1
2,RT capres jokowi wacapres abraham samad gub...,1
3,capres Prabowo and cawapres jokowi and...,1
4,RT capres Prabowo and cawapres jokowi ...,1


In [11]:
df_use.shape

(3356, 2)

In [13]:
# we will use 2500 for training and the rest for testing
X_train = df_use.loc[:2499, 'tweet'].values
y_train = df_use.loc[:2499, 'label'].values
X_test = df_use.loc[2500:, 'tweet'].values
y_test = df_use.loc[2500:, 'label'].values

In [15]:
X_train

array([' Gak setuju Jokowi jadi Cawapres   CAPRES JOKOWI harga mati   ',
       'capres jokowi wacapres abraham samad  gubernur ahok  gua yakin koruptor abissss  ',
       'RT   capres jokowi wacapres abraham samad  gubernur ahok  gua yakin koruptor abissss  ',
       ...,
       'Meski tak diusung sebagai capres  JK masih menjadi tokoh penting di dalam partai Golkar  JK sudah tak perlu partai lagi  tapi par   ',
       'Presiden Megawati punya ambisi',
       'Ditanya Soal Capres  Dahlan Lebih Pilih Menghindar  '],
      dtype=object)

In [14]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()

In [19]:
all_tweet = list_data_bersih
tokenizer.fit_on_texts(all_tweet) #tokenization

In [20]:

#pad sequence
max_length = max([len(s.split()) for s in all_tweet])

#vocab
vocab_length = len(tokenizer.word_index) + 1

X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

In [22]:
X_train_pad = pad_sequences(X_train_token, maxlen = max_length, padding = 'post')
X_test_pad = pad_sequences(X_test_token, maxlen = max_length, padding = 'post')

### Model Building

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

EMBEDDING_DIM = 128

model = Sequential()
model.add(Embedding(vocab_length, EMBEDDING_DIM, input_length = max_length))
model.add(GRU(units = 32, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = 'sigmoid'))

In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'] )

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 27, 128)           809856    
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                15456     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 825,345
Trainable params: 825,345
Non-trainable params: 0
_________________________________________________________________


In [28]:
#Training
model.fit(X_train_pad, y_train, batch_size=128, epochs=30, validation_data=(X_test_pad, y_test), verbose=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2500 samples, validate on 856 samples
Epoch 1/30
 - 2s - loss: 0.5184 - accuracy: 0.4508 - val_loss: 0.4004 - val_accuracy: 0.3843
Epoch 2/30
 - 1s - loss: 0.3428 - accuracy: 0.4552 - val_loss: 0.3607 - val_accuracy: 0.3843
Epoch 3/30
 - 1s - loss: 0.2023 - accuracy: 0.4552 - val_loss: 0.3515 - val_accuracy: 0.3843
Epoch 4/30
 - 1s - loss: -8.7222e+00 - accuracy: 0.4872 - val_loss: 0.5885 - val_accuracy: 0.4299
Epoch 5/30
 - 1s - loss: -6.3293e+01 - accuracy: 0.6256 - val_loss: 1.4990 - val_accuracy: 0.3633
Epoch 6/30
 - 1s - loss: -1.3916e+02 - accuracy: 0.6248 - val_loss: 2.0495 - val_accuracy: 0.3610
Epoch 7/30
 - 1s - loss: -2.0493e+02 - accuracy: 0.5936 - val_loss: 2.2588 - val_accuracy: 0.3621
Epoch 8/30
 - 1s - loss: -2.6778e+02 - accuracy: 0.6048 - val_loss: 2.3499 - val_accuracy: 0.3645
Epoch 9/30
 - 1s - loss: -3.3632e+02 - accuracy: 0.5892 - val_loss: 1.4699 - val_accuracy: 0.3621
Epoch 10/30
 - 1s - loss: -3.8584e+02 - accuracy: 0.5704 - val_loss: 1.4109 - val_accu